## TechGPT

05/06/2025
UNC Charlotte
ITCS 5010 : Design and Development of Generative AI Applications
Group 2: Cameron Detig, Zaid Jebril, Sri Girija Naga Anuhya Samudrala, Derek Smith

- Reads a database of prior user queries and llm responses.
- Uses queries to generate a new response using hidden chain of thought. 
- Evaluates the new response using ragas metrics.

Steps to use:

Create virtual environment and install everything in the requirements.txt

Create a .env file and add your DATABRICKS_TOKEN and DATABRICKS_HOST API keys.

In this directory add a .index vector database file, a .parquet metadata file, and a .csv file of test queries.
Assign their file names to the variables below.

In [6]:
index_file_path = "my_faiss.index"
metadata_file_path = "metadata.parquet"
test_queries_csv_path = 'Feedback_PHD_cleaned.csv'

In [7]:
from databricks_langchain import ChatDatabricks, DatabricksEmbeddings
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from ragas.metrics import context_precision, context_recall, faithfulness, answer_relevancy
from ragas.evaluation import evaluate
from datasets import Dataset
from dotenv import load_dotenv
import ast
import pandas as pd
import numpy as np
import faiss
import ragas
import re
import csv
import json
import os


load_dotenv(override=True)
os.environ['DATABRICKS_TOKEN'] = os.getenv('DATABRICKS_TOKEN')
os.environ['DATABRICKS_HOST'] = os.getenv('DATABRICKS_HOST')

# Define llms
chat_model = ChatDatabricks(
    endpoint="gpt-4o-mini",
    temperature=0.1,
    max_tokens=2000,
)

find_exact_keyword = ChatDatabricks(
    endpoint="gpt-4o-mini",
    temperature=0.1,
    max_tokens=2000,
)

# Define embedding model
embedding_model = DatabricksEmbeddings(
    endpoint = "ada-002"
)


# Load the FAISS index
faiss_index = faiss.read_index(index_file_path)
print("FAISS index loaded successfully!")

# Load metadata from the Parquet file
metadata_df = pd.read_parquet(metadata_file_path)
print("Metadata loaded successfully!")

FAISS index loaded successfully!
Metadata loaded successfully!


**Functions to be used by the callSystem function to run the response generation.**

In [8]:
def get_embeddings(embedding_model, query):
    '''Embeds the query using the specified embedding model.'''
    embedding = embedding_model.embed_query(query)
    return embedding


def retrieve_chunks(index, query, k=5):
    '''Retrieves the top k chunks from the FAISS index based on the query.'''
    query_vector = get_embeddings(embedding_model, query)
    query_vector = np.array(query_vector).astype("float32").reshape(1,-1)

    distance, indices = index.search(query_vector, k)

    retrieved_chunks = metadata_df.iloc[indices[0]].reset_index(drop=True)

    return retrieved_chunks


def rewrite_query(user_query):
    '''Rewrite the user's query to be more specific and concise'''

    rewrite_query_prompt = """
    You are an expert in understanding and rephrasing user service queries based on chat history. Your goal is to break down the user's intent and context into a concise, reformulated question. 

    Internal Thought Process:

    1. Analyze the user's current query {input}: Carefully examine the user's latest input to understand their core need and any specific details they have provided.
    2. Consider the chat history: Review the provided {chat_history} to identify any relevant prior interactions. Play close attention to the flow of the conversation and the assitant's previous responses. If there is no chat history or if the chat history is not related to the user's current query, consider the query as a standalone query. 
    3. Identify the user's underlying intent: Based on current query and relevant history, determine what the user is really trying to achieve or understand. 
    4. Generate a context-rich question: Formulate a new, standalone question that accurately reflect's the user's intent, incorporating relevant context from the chat history. Ensure that this generated question is not a direct follow-up or answer to the current user query. 
    5. Refine the question: Ensure that the new question is clear, concise, relevant and in a proper question format. Avoid any repetition or ambiguity. 

    Output: Only the reformulated user question should be provided below

    Based on the chat history: {chat_history}

    What is the user's core question, taking into account the user's previous conversation?
    """

    rewrite = PromptTemplate(
                        template = rewrite_query_prompt,
                        input_variables=["chat_history", "input"]
    )

    rewritten_question_chain = rewrite | chat_model
    rewritten_question = rewritten_question_chain.invoke({"chat_history": "", "input":user_query}).content

    return rewritten_question


def get_keywords_for_exact_search(user_query):
    '''Extracts keywords from the user's query for exact search.'''
       
    keyword_extraction_prompt = """

    # Advanced Keyword Extraction for Technical Queries

    You are an advanced AI assistant specialized in extracting crucial keywords from technical queries. Your expertise lies in pinpointing precise product information, including names, unique identifiers, and version details.
    Use the Input Query marked by the ****.

    ## Key Elements to Extract:

    - **Product Name:** The primary identifier or common name of a product (e.g., Experion PKS).
    - **SKU/Part Number:** Unique alphanumeric codes used to distinguish products or parts (e.g., RDYNAMO-8074, 1-EVDYZ2U).
    - **Version/Release Info:** Specific version numbers or release identifiers for products or documents (e.g., R2.2.1, Update4).

    ## Extraction Guidelines:

    - **Priority:** Focus on extracting the product name first, followed by any specific identifiers (SKU/Part Number) or version/release information.
    - **Patterns:** Pay attention to alphanumeric strings and version formats, which may include numbers, letters, hyphens, and periods.
    - **Ignore Non-essential Words:** Overlook filler words such as "is," "in," "for," etc., unless they are part of a product name or identifier.
    - **Special Cases:** Treat terms like "C300" or "301C" as product names when not accompanied by other identifiable product names.

    ## Task Execution:

    1. Carefully review each query to discern the primary focus.
    2. Extract relevant keywords, focusing on product names, part numbers/SKUs, and version/release information.
    3. Present the extracted keywords separated by commas. For queries that focus on a specific functionality or general inquiry without mentioning a product/version, use "no keyword found"

    ## Enhanced Examples:

    - **Input Query:** "Is there a Tools and Controller Update4 for R520.2?"
    - **Expected Keywords:** "Tools and Controller, Update4, R520.2"
    - **Input Query:** "What product anomalies are being introduced in Honeywell Forge Alarm Management Reporting R2.2.1?"
    - **Expected Keywords:** "Honeywell Forge Alarm Management Reporting, R2.2.1"
    - **Input Query:** "C300 sync steps for CBM6 v2"
    - **Expected Keywords:** "C300, CBM6 v2"
    - **Input Query:** "what is scada status in QuickBuilder?"
    - **Expected output:** "no keyword found"
    - **Input Query:** "C300 sync steps"
    - **Expected Keywords:** "C300"
    - **Input Query:** "What is a ‘ExpSQLAgtSvc’ account from local to domain users?"
    - **Expected Keywords:** "‘ExpSQLAgtSvc’"

    Input Query: ****{raw_query}****
    Keywords:
    """

    keywork_prompt = PromptTemplate(
        template = keyword_extraction_prompt,
        input_variables = ["raw_query"]
    ) 

    keyword_response_chain = keywork_prompt | chat_model
    keyword_reponse = keyword_response_chain.invoke({"raw_query": user_query}).content
    
    return keyword_reponse



def respond_using_documents(query, retrieved_docs, keywords=None):
    '''Generates a response using the retrieved documents and the user's query.'''

    prompt_prefix = """Respond to the user query marked by #### based on the sources encapsulated within ****. Make sure to pay attention to the keywords marked by >>>>.
    - Your response should strictly utilize the information from the provided Sources in line with user query.
    - Every piece of information or fact you use from a source must be immediately cited within the sentence itself, using its "fileName", DO NOT USE ANY OTHER NAME. Example: "The sky is blue [info.txt]."
    - DO NOT STATE CITATIONS at the very end of all the facts.
    - If a single fact is backed by multiple sources, integrate each citation within the same sentence: "The sky is blue [info1.txt][info2.txt]."
    - After formulating your answer, double-check for any inconsistencies or omitted citations.

    STRICTLY frame your response as a valid JSON using the following fields :
    - "response": The comprehensive answer to the user query with inline citations after each fact. For multiple lines, give a "\n" seperated answer.
    - "confidence": Quantify your confidence in the provided response on a numeric scale.
    - "reason": Briefly (in no more than 20 words) elucidate the rationale behind your answer.

    User query: ####{user_query}####

    Sources: ****{retr_docs}****

    Keywords: >>>>{keywords}>>>>
    
    Response: """ 

    final_prompt = PromptTemplate(
        template = prompt_prefix,
        input_variables = ["user_query", "retr_docs", "keywords"]
    ) 

    final_response_chain = final_prompt | chat_model
    final_reponse = final_response_chain.invoke({"user_query": query, "retr_docs": retrieved_docs, "keywords": keywords}).content


    # Clean and parse the JSON
    try:
        final_reponse = final_reponse.strip('```json').strip()
        cleaned_response = final_reponse.strip('```').strip()
        data = json.loads(cleaned_response)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
        print("Raw response:", final_reponse)

    # Extract the desired fields
    response = data.get("response", "")
    confidence = data.get("confidence", 0)
    reason = data.get("reason", "")

    return response, confidence, reason


**callSystem() function to manage the response generation pipeline**

In [9]:
def callSystem(query):
    '''Main function to handle the user query and generate a response.'''

    # User inputted query
    print("\nUser Query: ")
    print(query)

    # Extract keywords from the user query for exact search
    keywords = get_keywords_for_exact_search(query)
    print("\nExtracted Keywords: ")
    print(keywords)

    # Rewrite the query to be more specific and concise
    rewritten_question = rewrite_query(query)   
    print("\nRewritten Query: ")
    print(rewritten_question)


    # Use the rewritten question to search the vector store for relevant documents 
    retr_docs = retrieve_chunks(faiss_index, rewritten_question)['file_content'].to_list()

    # Use the rewritten question and retrieved documents to generate a response
    response, confidence, reason = respond_using_documents(rewritten_question, retr_docs, keywords)

    # Print the extracted values
    print("\n\nResponse:")
    print(response)
    print("\nConfidence:")
    print(confidence)
    print("\nReason:")
    print(reason)

    return response, confidence, reason


# For Testing the system
#query = "Do you know of a method, which does not involve a total rebuild, for changing the ‘ExpSQLSvc’ and ‘ExpSQLAgtSvc’ accounts from local to domain users, please?"
#callSystem(query)


**Generate answers for all queries in the csv and export to new csv named "Output_System_Responses.csv"**

In [ ]:
# Load the CSV file of test queries into a DataFrame. test_queries_csv_path is set in the first cell of the notebook.
queries_df = pd.read_csv(test_queries_csv_path, encoding='latin1')

# The number of queries to process, set as high as needed.
query_limit = 5

# Path to the output CSV file
responses_csv_file_path = 'Output_System_Responses.csv'

# Write to the CSV file row by row
with open(responses_csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Index', 'casenumber', 'User_Query', "System_Response", "ai_answer"])
    writer.writeheader()  # Write the header row

    for index, row in queries_df.iterrows():
        if index < query_limit:
            print("\nQuery: " + str(index) + "  -----------------------------------------")
            response = callSystem(row['User_Query'])
            writer.writerow({"Index": index, "casenumber": row["casenumber"], "User_Query": row['User_Query'], "System_Response": response[0], "ai_answer": row["ai_answer"]})  # Write each row

print(f"Data has been written to {responses_csv_file_path}")

**Load "Output_System_Responses.csv" and use it to evaluate the responses using RAGAs. Export results to "Output_RAGAs.csv"**

In [ ]:
# Maximum number of responses to evaluate
evaluation_limit = 1000

evaluation_model = ChatDatabricks(
    endpoint="gpt-4o-mini",
    temperature=0.1,
    max_tokens=2000,
)

# Load the CSV file
df = pd.read_csv(responses_csv_file_path, encoding='latin1')

# Path to the output CSV file
output_csv_file_path = 'Output_RAGAs.csv'

# Write to the CSV file row by row
with open(output_csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Index', 'casenumber', 'User_Query', "System_Response", "Original_AI_Answer",
                                              "Faithfulness", "Answer_Relevancy", "Context_Recall", "Context_Precision"])
    writer.writeheader()  # Write the header row

    for index, row in df.iterrows():
        if index < evaluation_limit:

            # Embed the query
            query_vector_embedding = get_embeddings(embedding_model, row['User_Query'])
            query_vector = np.array(query_vector_embedding).astype("float32").reshape(1,-1)
            
            # Use vector of query to search faiss for relevant docs
            k=5
            distance, indices = faiss_index.search(query_vector, k)
            retrieved_chunks = metadata_df.iloc[indices[0]].reset_index(drop=True)


            # Arrange information into a dictionary
            data = {
                "question": [row['User_Query']],
                "answer": [row['System_Response']],
                "contexts": [retrieved_chunks['file_content'].tolist()],
                "ground_truth": [row["ai_answer"]]
            }

            print(f"\nIndex: {index}-----------------------------------------")
            print("\nQuestion:")
            print(data["question"][0])
            print("\nResponse:")
            print(data["answer"][0])
            print("\nGround_Truth:")
            print(data["ground_truth"][0])
            
            # Convert dictionary into dataset
            dataset = Dataset.from_dict(data)

            # Evaluate with RAGAS
            result = evaluate(
                dataset = dataset,
                metrics= [faithfulness, answer_relevancy, context_recall, context_precision],
                llm = evaluation_model
            )

            print("RAGAS Score:", result)

            writer.writerow({"Index": index, 
                             "casenumber": row["casenumber"], 
                             "User_Query": row['User_Query'], 
                             "System_Response": row["System_Response"], 
                             "Original_AI_Answer": row["ai_answer"],
                             "Faithfulness":  round(result["faithfulness"][0], 2), 
                             "Answer_Relevancy": round(result["answer_relevancy"][0], 2),
                             "Context_Recall": round(result["context_recall"][0], 2), 
                             "Context_Precision": round(result["context_precision"][0], 2)})  # Write each row